In [32]:
import pandas as pd
import numpy as np
import collections
from sklearn.tree import DecisionTreeClassifier

In [2]:
#Lendo conjuntos de teste e treino
train_validation = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
def train_validate_split(df, train_percent=.7, validate_percent=.3, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    return train, validate

In [10]:
#Divide o dataset de treino entre treino e validação (70% e 30%)
train, validate = train_validate_split(train_validation)

In [14]:
print("Full train:" + str(len(train_validation)) + ", train:" + str(len(test)) + ", validate:" + str(len(validate)))

Full train:595212, train:892816, validate:178563


In [15]:
#Dez primeiras linhas do conjunto de treino
train.head(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
216406,540636,0,1,1,8,0,6,0,0,1,...,5,3,1,8,0,0,1,0,0,1
261535,653525,0,2,1,6,1,0,0,1,0,...,7,2,2,8,0,1,0,1,0,0
301082,752313,0,1,1,1,0,0,1,0,0,...,2,2,3,8,0,1,1,0,1,0
401214,1002620,0,2,1,5,0,0,1,0,0,...,8,3,3,7,0,1,1,0,0,0
432498,1080743,0,0,1,1,0,0,0,0,0,...,10,0,3,9,0,0,1,0,0,1
85591,214076,0,2,1,2,1,0,1,0,0,...,7,1,3,8,0,1,1,0,0,0
211823,529200,0,1,1,3,1,0,0,1,0,...,6,3,3,6,0,1,1,1,0,1
109565,274309,0,5,1,5,1,4,1,0,0,...,7,1,3,13,0,1,1,1,1,0
148629,371572,1,1,1,5,1,0,0,1,0,...,3,1,3,8,0,0,1,0,1,0
36271,90907,0,2,1,3,1,0,0,1,0,...,8,1,2,5,0,1,0,1,0,0


In [16]:
#Dez primeiras linhas do conjunto de treino
test.head(10)

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1
5,5,0,1,6,0,0,1,0,0,0,...,8,1,4,9,1,0,1,0,1,0
6,6,0,1,3,0,0,0,1,0,0,...,2,0,4,6,1,1,0,0,0,0
7,8,0,1,0,0,0,1,0,0,0,...,3,1,4,9,0,1,0,0,0,0
8,10,0,1,7,0,0,0,1,0,0,...,5,1,4,6,0,0,1,0,0,0
9,11,1,1,6,0,0,0,0,0,1,...,6,1,6,10,0,1,1,0,0,0


In [17]:
#Mostrando a lista dos headers
list(test)

['id',
 'ps_ind_01',
 'ps_ind_02_cat',
 'ps_ind_03',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'ps_calc_01',
 'ps_calc_02',
 'ps_calc_03',
 'ps_calc_04',
 'ps_calc_05',
 'ps_calc_06',
 'ps_calc_07',
 'ps_calc_08',
 'ps_calc_09',
 'ps_calc_10',
 'ps_calc_11',
 'ps_calc_12',
 'ps_calc_13',
 'ps_calc_14',
 'ps_calc_15_bin',
 'ps_calc_16_bin',
 'ps_calc_17_bin',
 'ps_calc_18_bin',
 'ps_calc_19_bin',
 'ps_calc_20_bin']

In [12]:
#Criação de novo DataFrame a partir de One-Hot encoding
new_data_train = pd.get_dummies(
    data=train, 
    prefix=['ps_ind_02_cat_encoded', 'ps_ind_04_cat_encoded',
            'ps_ind_05_cat_encoded', 'ps_car_01_cat_encoded',
            'ps_car_02_cat_encoded', 'ps_car_03_cat_encoded',
            'ps_car_04_cat_encoded', 'ps_car_05_cat_encoded',
            'ps_car_06_cat_encoded', 'ps_car_07_cat_encoded',
            'ps_car_08_cat_encoded', 'ps_car_09_cat_encoded',
            'ps_car_10_cat_encoded', 'ps_car_11_cat_encoded'], 
    columns=['ps_ind_02_cat', 'ps_ind_04_cat',
            'ps_ind_05_cat', 'ps_car_01_cat',
            'ps_car_02_cat', 'ps_car_03_cat',
            'ps_car_04_cat', 'ps_car_05_cat',
            'ps_car_06_cat', 'ps_car_07_cat',
            'ps_car_08_cat', 'ps_car_09_cat',
            'ps_car_10_cat', 'ps_car_11_cat'])

#Criação de novo DataFrame a partir de One-Hot encoding
new_data_test = pd.get_dummies(
    data=test, 
    prefix=['ps_ind_02_cat_encoded', 'ps_ind_04_cat_encoded',
            'ps_ind_05_cat_encoded', 'ps_car_01_cat_encoded',
            'ps_car_02_cat_encoded', 'ps_car_03_cat_encoded',
            'ps_car_04_cat_encoded', 'ps_car_05_cat_encoded',
            'ps_car_06_cat_encoded', 'ps_car_07_cat_encoded',
            'ps_car_08_cat_encoded', 'ps_car_09_cat_encoded',
            'ps_car_10_cat_encoded', 'ps_car_11_cat_encoded'], 
    columns=['ps_ind_02_cat', 'ps_ind_04_cat',
            'ps_ind_05_cat', 'ps_car_01_cat',
            'ps_car_02_cat', 'ps_car_03_cat',
            'ps_car_04_cat', 'ps_car_05_cat',
            'ps_car_06_cat', 'ps_car_07_cat',
            'ps_car_08_cat', 'ps_car_09_cat',
            'ps_car_10_cat', 'ps_car_11_cat'])

new_data_train.head(10)

,id,target,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,...,ps_car_11_cat_encoded_95,ps_car_11_cat_encoded_96,ps_car_11_cat_encoded_97,ps_car_11_cat_encoded_98,ps_car_11_cat_encoded_99,ps_car_11_cat_encoded_100,ps_car_11_cat_encoded_101,ps_car_11_cat_encoded_102,ps_car_11_cat_encoded_103,ps_car_11_cat_encoded_104
0,7,0,2,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,1,7,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13,0,5,9,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,19,0,5,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,20,0,2,3,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,22,0,5,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,26,0,5,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,28,1,1,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
#Proporção de target 1 e 0
#new_data_train['target'].value_counts()
train['target'].value_counts()

0    401583
1     15065
Name: target, dtype: int64

In [19]:
#Separando features e target para criação do modelo
#x = new_data_train.drop('target', axis=1)
#y = new_data_train['target']
x = train.drop('target', axis=1)
y = train['target']

In [20]:
#Features não contém o campo target
x.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
216406,540636,1,1,8,0,6,0,0,1,0,...,5,3,1,8,0,0,1,0,0,1
261535,653525,2,1,6,1,0,0,1,0,0,...,7,2,2,8,0,1,0,1,0,0
301082,752313,1,1,1,0,0,1,0,0,0,...,2,2,3,8,0,1,1,0,1,0
401214,1002620,2,1,5,0,0,1,0,0,0,...,8,3,3,7,0,1,1,0,0,0
432498,1080743,0,1,1,0,0,0,0,0,1,...,10,0,3,9,0,0,1,0,0,1


In [21]:
#Features não contém o campo target
y.head()

216406    0
261535    0
301082    0
401214    0
432498    0
Name: target, dtype: int64

In [22]:
#Criação do modelo e treino
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [23]:
#Verificando score no conjunto de treino
clf.score(x, y)

0.96385438067625429

In [26]:
#Verificando score no conjunto de validação
x_validate = validate.drop('target', axis=1)
y_validate = validate['target']
clf.score(x_validate, y_validate)

0.96287584774001334

In [33]:
#Array de resposta
collections.Counter(clf.predict(test))

Counter({0: 892775, 1: 41})

In [29]:
#Você precisa enviar ao Kaggle um arquivo .csv com as suas previsões. 
#Esse arquivo deve conter duas colunas com cabeçalho: id e target.
submission = pd.DataFrame()
submission['id'] = test['id']
submission['target'] = clf.predict(test)
submission.to_csv('submission.csv', index=False)